In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
import pandas as pd
import json

In [0]:
class Check_HR:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.data_json = {"Data":[]}    
  
  def logger(message):
    print(message)
    
  def set_query_string(self):
    query = """MERGE INTO com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed process_table 
USING (
  select tb1.NGEBOX_RECORD_ID,tb2.id, tb1.suggest_external_id_vod__c,tb4.hist_retention from com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed tb1
  inner join com_us_alyt_omnichannel.ib_ilay_rep_account tb2 on tb1.account_vod__c = tb2.id and tb2.TKD_HR_HOLD__C = 1
  left join com_us_lake.rep_suggestion_vod__c tb3 on tb3.SUGGESTION_EXTERNAL_ID_VOD__C = tb1.suggest_external_id_vod__c
  inner join com_us_alyt_ngebox.metadata_usecase tb4 on tb1.usecase_id = tb4.usecase_id
  where (tb3.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) 
or (tb3.SUGGESTION_EXTERNAL_ID_VOD__C is not null and tb3.DISMISSED_VOD__C = 0) 
) source_table ON (process_table.account_vod__c = source_table.id and process_table.NGEBOX_RECORD_ID = source_table.NGEBOX_RECORD_ID and  process_table.suggest_external_id_vod__c = source_table.suggest_external_id_vod__c and CAST(process_table.created_date as DATE) >= date_add(current_date,-source_table.hist_retention))
WHEN MATCHED THEN
UPDATE SET process_table.HR_Hold_Flag = 1,
process_table.Not_Sent_To_Veeva_Reason = case  when ISNULL(process_table.Not_Sent_To_Veeva_Reason)= true then 'Record cannot be loaded due to HR Hold' else concat(Not_Sent_To_Veeva_Reason, '|', 'Record cannot be loaded due to HR Hold' ) end;"""
    self.query = query    

  def run_query_and_set_sqldf(self):
    query_list = self.query.split(";")
    self.sql_df = []
    for curr_query in query_list:
      if curr_query.strip() == "":
        continue
      sql_df = sqlContext.sql(curr_query).collect() # type(sql_df) -> dict
      self.sql_df = self.sql_df + sql_df
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    nb_json = {"data":sql_jsonparse} # type(metadata_json) -> dict
    self.nb_json = nb_json
    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.nb_json
      }
    }
    self.response = resp

In [0]:
nb_obj = Check_HR()
nb_obj.set_query_string()
nb_obj.run_query_and_set_sqldf()
nb_obj.format_sqldf_to_json()
nb_obj.construct_microservice_response()
dbutils.notebook.exit(
  nb_obj.response
)

{'nge_response': {'status': 200, 'body': {'data': [{'num_affected_rows': 0, 'num_updated_rows': 0, 'num_deleted_rows': 0, 'num_inserted_rows': 0}]}}}